In [29]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
from tqdm import tqdm
import torch
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# 파일 경로 설정 (목록 확인 후 정확한 파일 이름 사용)
admissions_path = '/data/MIMIC_IV_private/mimic-iv-3.0/hosp/admissions.csv'
diagnoses_icd_path = '/data/MIMIC_IV_private/mimic-iv-3.0/hosp/diagnoses_icd.csv'
labevents_path = '/data/MIMIC_IV_private/mimic-iv-3.0/hosp/labevents.csv'
prescriptions_path = '/data/MIMIC_IV_private/mimic-iv-3.0/hosp/prescriptions.csv'
d_icd_diagnoses_path = '/data/MIMIC_IV_private/mimic-iv-3.0/hosp/d_icd_diagnoses.csv'
icustays_path = '/data/MIMIC_IV_private/mimic-iv-3.0/icu/icustays.csv'
procedureevents_path = '/data/MIMIC_IV_private/mimic-iv-3.0/icu/procedureevents.csv'


# 데이터 불러오기
try:
    admissions = pd.read_csv(admissions_path)
    diagnoses_icd = pd.read_csv(diagnoses_icd_path)
    labevents = pd.read_csv(labevents_path)
    procedureevents = pd.read_csv(procedureevents_path)
    prescriptions = pd.read_csv(prescriptions_path)
    d_icd_diagnoses = pd.read_csv(d_icd_diagnoses_path)
    icustays = pd.read_csv(icustays_path)
    
    # 데이터 일부 확인
    print(admissions.head())
    #print(patients.head())
    print(diagnoses_icd.head())
    print(labevents.head())
    print(procedureevents.head())
    print(prescriptions.head())
    print(d_icd_diagnoses.head())
    print(icustays.head())

except FileNotFoundError as e:
    print(f"Error: {e}")

/tmp/ipykernel_1234443/1629963062.py:17: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv(prescriptions_path)


   subject_id   hadm_id            admittime            dischtime deathtime  \
0    10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00       NaN   
1    10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00       NaN   
2    10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00       NaN   
3    10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00       NaN   
4    10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P49AFC  TRANSFER FROM HOSPITAL   
1        EW EMER.            P784FA          EMERGENCY ROOM   
2        EW EMER.            P19UTS          EMERGENCY ROOM   
3        EW EMER.            P06OTX          EMERGENCY ROOM   
4  EU OBSERVATION            P39NWO          EMERGENCY ROOM   

  discharge_location insurance language marital_status   race  \
0               HOME  Medicaid  English        WIDOWED  WHITE   
1               

### 패혈증 필터링

In [30]:
# 패혈증 관련 진단명을 포함하는 행 필터링
sepsis_conditions = d_icd_diagnoses[d_icd_diagnoses['long_title'].str.contains('Sepsis', case=False) | 
                                    d_icd_diagnoses['long_title'].str.contains('Septicemia', case=False)]

# 필터링된 진단 코드 확인
print("Sepsis-related Diagnoses Codes:")
print(sepsis_conditions)

Sepsis-related Diagnoses Codes:
      icd_code  icd_version                                         long_title
9         0031            9                              Salmonella septicemia
517       0223            9                                 Anthrax septicemia
578       0380            9                           Streptococcal septicemia
579      03810            9             Staphylococcal septicemia, unspecified
580      03811            9  Methicillin susceptible Staphylococcus aureus ...
...        ...          ...                                                ...
43985    R6521           10                    Severe sepsis with septic shock
96313    T8144           10                       Sepsis following a procedure
96314  T8144XA           10    Sepsis following a procedure, initial encounter
96315  T8144XD           10  Sepsis following a procedure, subsequent encou...
96316  T8144XS           10              Sepsis following a procedure, sequela

[82 rows x 3 column

In [31]:
# diagnoses_icd와 결합하여 패혈증 환자 식별
sepsis_id = diagnoses_icd[diagnoses_icd['icd_code'].isin(sepsis_conditions['icd_code'])].reset_index()

# 결과 확인 및 패혈증 관련 진단 코드의 빈도 확인
print("Sepsis ICD Code Frequency:")
print(sepsis_id['icd_code'].value_counts())

Sepsis ICD Code Frequency:
icd_code
A419       7770
R6521      5599
99592      5257
0389       4773
99591      2953
R6520      1948
A4151      1413
A4189      1338
03842      1007
A4101       641
0380        635
03849       627
A4159       558
A4181       539
03811       490
A4102       334
03812       297
T8144XA     267
A4152       249
A408        240
A411        224
0388        181
A4150       178
03843       178
03819       166
B377        145
0383        132
A401        125
0382        106
A414         99
03840        96
A403         67
A4153        65
A400         60
03844        51
03810        31
A021         27
A409         24
O85          20
A413         20
A412         14
O0337        11
67022        10
03841         8
67024         8
0031          7
O8604         6
A427          5
A327          5
A5486         4
O0882         3
0545          1
A227          1
O0387         1
T8144XD       1
O0487         1
Name: count, dtype: int64


In [32]:
sepsis_id

,index,subject_id,hadm_id,seq_num,icd_code,icd_version
0,202,10000826,21086876,2,99591,9
1,612,10001401,27012892,2,A4181,10
2,613,10001401,27012892,3,R6520,10
3,617,10001401,27012892,7,T8144XA,10
4,628,10001401,27060146,2,A419,10
...,...,...,...,...,...,...
39011,6362208,19997367,20617667,4,99592,9
39012,6362258,19997367,21508795,1,0380,9
39013,6362275,19997367,21508795,18,99592,9
39014,6363173,19997886,20793010,4,A419,10


In [33]:
sepsis_id=sepsis_id[sepsis_id['icd_version']==9]

In [34]:
sepsis_id

,index,subject_id,hadm_id,seq_num,icd_code,icd_version
0,202,10000826,21086876,2,99591,9
7,1207,10002013,24848509,1,0389,9
8,1211,10002013,24848509,5,99592,9
10,1736,10002428,23473524,1,03843,9
11,1741,10002428,23473524,6,99591,9
...,...,...,...,...,...,...
39008,6361529,19995732,21415030,4,99591,9
39010,6362207,19997367,20617667,3,03849,9
39011,6362208,19997367,20617667,4,99592,9
39012,6362258,19997367,21508795,1,0380,9


In [35]:
# 상위 5개의 패혈증 ICD 코드를 선택
top_n = 5
top_sepsis_codes = sepsis_id['icd_code'].value_counts().index[:top_n]

# 상위 5개의 코드에 해당하는 환자 필터링
sepsis = sepsis_id[sepsis_id['icd_code'].isin(top_sepsis_codes)].drop(['index', 'seq_num','icd_version'], axis=1).reset_index(drop=True)

# 환자의 ID만 추출하여 환자id에 저장
환자id = sepsis['subject_id'].unique()

In [36]:
top_sepsis_codes

Index(['99592', '0389', '99591', '03842', '0380'], dtype='object', name='icd_code')

In [37]:
sepsis

,subject_id,hadm_id,icd_code
0,10000826,21086876,99591
1,10002013,24848509,0389
2,10002013,24848509,99592
3,10002428,23473524,99591
4,10002428,28662225,99592
...,...,...,...
14620,19995732,21415030,0389
14621,19995732,21415030,99591
14622,19997367,20617667,99592
14623,19997367,21508795,0380


In [38]:
환자id

array([10000826, 10002013, 10002428, ..., 19992202, 19995732, 19997367])

# label.csv 만들기

In [39]:
# 환자id 리스트를 이용하여 입퇴원 필터링
입퇴원 = icustays[icustays['subject_id'].isin(환자id)].copy()

# datetime 형식으로 변환
입퇴원['intime'] = pd.to_datetime(입퇴원['intime'])
입퇴원['outtime'] = pd.to_datetime(입퇴원['outtime'])
admissions['deathtime'] = pd.to_datetime(admissions['deathtime'])

# 필요한 열만 추출
입퇴원_subset = 입퇴원[['subject_id', 'outtime', 'los']]
admissions_subset = admissions[['subject_id', 'deathtime']]

입퇴원_subset = 입퇴원_subset[입퇴원_subset['los'] >= 3].copy()

# 시간 정보에서 시간을 제거하여 날짜 형식으로 변경
입퇴원_subset['outtime'] = 입퇴원_subset['outtime'].dt.date
admissions_subset.loc[:, 'deathtime'] = admissions_subset['deathtime'].dt.date

# admissions_cleaned 처리 및 필터링
admissions_cleaned = admissions_subset.sort_values(by='deathtime', na_position='last').drop_duplicates(subset='subject_id', keep='first')
admissions_cleaned = admissions_cleaned.sort_values(by='subject_id', ascending=True)

# 입퇴원_subset의 unique subject_id에 대해 admissions_filtered 필터링
unique_subject_ids = 입퇴원_subset['subject_id'].unique()
admissions_filtered = admissions_cleaned[admissions_cleaned['subject_id'].isin(unique_subject_ids)]

# Merge 입퇴원_subset과 admissions_filtered, deathtime 추가
merged_df = 입퇴원_subset.merge(admissions_filtered[['subject_id', 'deathtime']], on='subject_id', how='left')

# 'los' 열 삭제
merged_df = merged_df.drop(columns=['los'])

# 'outtime'을 'discharge'로 이름 변경
merged_df = merged_df.rename(columns={'outtime': 'discharge'})

# 'discharge' 및 'deathtime'이 모두 datetime.date 형식인지 확인
merged_df['discharge'] = pd.to_datetime(merged_df['discharge']).dt.date
merged_df['deathtime'] = pd.to_datetime(merged_df['deathtime']).dt.date

# 'LABEL_DEAD1_ALIVE0' 생성
merged_df['LABEL_DEAD1_ALIVE0'] = merged_df.apply(
    lambda row: 1 if pd.notna(row['deathtime']) and row['deathtime'] <= row['discharge'] else 0,
    axis=1
)

# 'deathtime' 열 삭제
merged_df = merged_df.drop(columns=['deathtime'])

In [19]:
# 환자id 리스트를 이용하여 입퇴원 필터링
입퇴원 = icustays[icustays['subject_id'].isin(환자id)].copy()

# datetime 형식으로 변환
입퇴원['intime'] = pd.to_datetime(입퇴원['intime'])
입퇴원['outtime'] = pd.to_datetime(입퇴원['outtime'])
admissions['deathtime'] = pd.to_datetime(admissions['deathtime'])

# 필요한 열만 추출
입퇴원_subset = 입퇴원[['subject_id', 'intime', 'outtime', 'los']]
admissions_subset = admissions[['subject_id', 'deathtime']]

# icustays와 admissions 데이터 병합 (left join으로 icustays 기준)
merged_df = 입퇴원_subset.merge(admissions_subset, on='subject_id', how='left')

# 중복 제거: deathtime이 있는 행을 우선적으로 남기기
merged_df['has_deathtime'] = ~merged_df['deathtime'].isna()

merged_df = merged_df.sort_values(by=['has_deathtime', 'intime'], ascending=[False, True])
merged_df = merged_df.drop_duplicates(subset=['subject_id', 'intime', 'outtime'], keep='first')


# 'los'가 3일 이상인 데이터만 필터링
#merged_df = merged_df[merged_df['los'] >= 3].copy()

#los 5
merged_df = merged_df[merged_df['los'] >= 5].copy()


# 시간 정보에서 시간을 제거하여 날짜 형식으로 변경
merged_df['intime'] = merged_df['intime'].dt.date
merged_df['outtime'] = merged_df['outtime'].dt.date
merged_df['deathtime'] = merged_df['deathtime'].dt.date

# 새로운 라벨 열 생성
def create_label(row):
    if pd.isnull(row['deathtime']):
        return 0  # 사망 시간 정보가 없으면 생존
    elif row['deathtime'] <= row['outtime']:
        return 1  # 퇴원 시간과 같거나 이전에 사망했으면 사망
    else:
        return 0  # 퇴원 이후 사망하지 않았으면 생존

# apply 함수를 사용하여 라벨 생성
merged_df['LABEL_DEAD1_ALIVE0'] = merged_df.apply(create_label, axis=1)

# 'intime', 'deathtime', 'los' 열 제거 및 'outtime'을 'discharge'로 이름 변경
merged_df = merged_df.drop(columns=['intime', 'deathtime', 'los','has_deathtime'])
merged_df = merged_df.rename(columns={'outtime': 'discharge'})

# subject_id와 discharge 기준으로 정렬
merged_df = merged_df.sort_values(by=['subject_id', 'discharge'])

In [40]:
merged_df

,subject_id,discharge,LABEL_DEAD1_ALIVE0
0,10002428,2156-05-22,0
1,10002428,2156-04-17,0
2,10002428,2156-04-26,0
3,10004235,2196-02-29,0
4,10004401,2144-04-09,0
...,...,...,...
5007,19986880,2185-07-14,0
5008,19986880,2185-08-10,1
5009,19991446,2141-09-24,0
5010,19997367,2126-05-16,0


In [41]:
#merged_df.to_csv('label.csv')

merged_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_9)/label.csv', index = False)

# Data.csv

#### 4.1 패혈증 환자 추출
##### labevents에서 패혈증환자 id만을 추출하여 변수 lab에 저장

In [42]:
# 환자 ID에 해당하는 labevents 데이터 필터링
lab = labevents[labevents['subject_id'].isin(환자id)].reset_index(drop=True)

# value_counts()를 위해 결측치를 문자열 'nan'으로 설정
lab['flag'] = lab['flag'].fillna('nan')

# 결과 확인
print("Filtered Lab Events Data:")
print(lab.head())

print("Value Counts for FLAG column:")
print(lab['flag'].value_counts())

Filtered Lab Events Data:
   labevent_id  subject_id  hadm_id  specimen_id  itemid order_provider_id  \
0         4235    10000826      NaN     17890668   51237               NaN   
1         4236    10000826      NaN     17890668   51274               NaN   
2         4237    10000826      NaN     17890668   51275               NaN   
3         4238    10000826      NaN     58045060   50955               NaN   
4         4239    10000826      NaN     59073038   50856               NaN   

             charttime            storetime value  valuenum valueuom  \
0  2146-12-05 11:30:00  2146-12-05 12:54:00   1.3       1.3      NaN   
1  2146-12-05 11:30:00  2146-12-05 12:54:00  15.2      15.2      sec   
2  2146-12-05 11:30:00  2146-12-05 12:54:00  26.0      26.0      sec   
3  2146-12-05 11:30:00                  NaN   ___       NaN      NaN   
4  2146-12-05 11:30:00  2146-12-06 00:46:00   NaN       NaN    ug/mL   

   ref_range_lower  ref_range_upper      flag priority comments  
0     

#### 4.2 lab

In [43]:
#필터링된 lab 데이터 생성
lab2 = lab.copy()
lab2 = lab2[['subject_id', 'itemid', 'charttime', 'flag']]
lab2 = lab2[lab2['flag'] == 'abnormal']

# charttime의 분과 초를 0으로 설정
lab2['charttime'] = pd.to_datetime(lab2['charttime'])
lab2['charttime'] = lab2['charttime'].apply(lambda x: x.replace(minute=0, second=0))

# 새로운 열 추가
lab2['type'] = 'LAB'

In [44]:
lab2

,subject_id,itemid,charttime,flag,type
0,10000826,51237,2146-12-05 11:00:00,abnormal,LAB
1,10000826,51274,2146-12-05 11:00:00,abnormal,LAB
5,10000826,50861,2146-12-05 11:00:00,abnormal,LAB
6,10000826,50862,2146-12-05 11:00:00,abnormal,LAB
7,10000826,50863,2146-12-05 11:00:00,abnormal,LAB
...,...,...,...,...,...
15475085,19997367,51244,2128-10-14 11:00:00,abnormal,LAB
15475087,19997367,51249,2128-10-14 11:00:00,abnormal,LAB
15475090,19997367,51256,2128-10-14 11:00:00,abnormal,LAB
15475091,19997367,51265,2128-10-14 11:00:00,abnormal,LAB


In [45]:
# lab의 SUBJECT_ID 열을 활용해 데이터 인덱싱
sub_list = lab['subject_id'].unique()

In [46]:
sub_list

array([10000826, 10002013, 10002428, ..., 19992202, 19995732, 19997367])

#### 4.3 pre

In [47]:
# prescriptions 데이터프레임을 복사하여 pre2 생성
pre2 = prescriptions.copy()

# 환자 ID가 sub_list에 포함된 행 필터링
pre2 = pre2[pre2['subject_id'].isin(sub_list)]

# 필요한 열만 선택
pre2 = pre2[['subject_id', 'starttime', 'stoptime', 'ndc']]

# starttime과 stoptime이 모두 결측치인 행의 인덱스
both_null = pre2[(pre2['starttime'].isnull()) & (pre2['stoptime'].isnull())].index

# ndc가 결측치인 행의 인덱스
ndc_null = pre2[pre2['ndc'].isnull()].index

pre2 = pre2.drop(index=both_null)
pre2 = pre2.drop(index=ndc_null)

# stoptime이 결측치인 경우 starttime으로 대체
pre2.loc[pre2['stoptime'].isnull(), 'stoptime'] = pre2.loc[pre2['stoptime'].isnull(), 'starttime']

# starttime이 결측치이고 stoptime이 존재하는 경우 stoptime으로 대체
pre2.loc[(pre2['starttime'].isnull()) & (pre2['stoptime'].notnull()), 'starttime'] = pre2.loc[(pre2['starttime'].isnull()) & (pre2['stoptime'].notnull()), 'stoptime']

# subject_id와 starttime을 기준으로 정렬
pre2 = pre2.sort_values(['subject_id', 'starttime']).reset_index(drop=True)

In [48]:
pre2

,subject_id,starttime,stoptime,ndc
0,10000826,2146-12-05 20:00:00,2146-12-06 08:00:00,6.332303e+10
1,10000826,2146-12-05 20:00:00,2146-12-06 08:00:00,3.380043e+08
2,10000826,2146-12-05 23:00:00,2146-12-12 21:00:00,0.000000e+00
3,10000826,2146-12-06 00:00:00,2146-12-06 08:00:00,4.091258e+08
4,10000826,2146-12-06 00:00:00,2146-12-06 08:00:00,4.091762e+08
...,...,...,...,...
2427907,19997367,2128-02-24 07:00:00,2128-02-25 06:00:00,6.155301e+10
2427908,19997367,2128-02-25 09:00:00,2128-02-26 08:00:00,2.450041e+08
2427909,19997367,2128-02-26 03:00:00,2128-02-26 20:00:00,9.043991e+08
2427910,19997367,2128-02-26 10:00:00,2128-02-26 20:00:00,2.450041e+08


In [49]:
def date_range(start, end):
    total_hours = int((end - start).total_seconds() // 3600) + 1
    dates = [(start + timedelta(hours=i)).replace(minute=0, second=0, microsecond=0) for i in range(total_hours)]
    return dates

In [50]:
# pre2를 리스트로 변환
pre2_list = pre2.values.tolist()
pre2_dic = {}
lst_time = []
lst_itemid = []
lst_subid = []

# pre2_list의 각 원소(즉, 각 처방에 대한 정보)를 순회합니다.
for idx, row in enumerate(tqdm(pre2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    # 시작 날짜와 종료 날짜를 datetime 형식으로 변환
    start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
    
    # 날짜 범위 생성
    day_list = date_range(start, end)
    
    # index에 맞춰서 subid, date, itemid를 넣어주기 위한 dic
    pre2_dic[idx] = day_list
    
    # pre2 DATE열에 들어갈 리스트
    lst_time.extend(pre2_dic[idx][:])

    # pre2 itemid(NDC)열에 들어갈 리스트
    for _ in pre2_dic[idx]:
        lst_itemid.append(itemid)
    
    # pre2 subid열에 들어갈 리스트
    for _ in pre2_dic[idx]:
        lst_subid.append(sub)
        

100%|███████████████████████████████| 2427912/2427912 [05:39<00:00, 7145.23it/s]


In [51]:
pre2sub = pd.DataFrame(lst_subid)
pre2time = pd.DataFrame(lst_time)
pre2item = pd.DataFrame(lst_itemid)

pre2 = pd.concat([pre2sub, pre2time, pre2item], axis = 1)
pre2.columns = ['subject_id', 'charttime', 'itemid']
pre2['type'] = 'pre'

In [52]:
pre2

,subject_id,charttime,itemid,type
0,10000826,2146-12-05 20:00:00,6.332303e+10,pre
1,10000826,2146-12-05 21:00:00,6.332303e+10,pre
2,10000826,2146-12-05 22:00:00,6.332303e+10,pre
3,10000826,2146-12-05 23:00:00,6.332303e+10,pre
4,10000826,2146-12-06 00:00:00,6.332303e+10,pre
...,...,...,...,...
182746874,19997367,2128-02-26 16:00:00,7.643903e+10,pre
182746875,19997367,2128-02-26 17:00:00,7.643903e+10,pre
182746876,19997367,2128-02-26 18:00:00,7.643903e+10,pre
182746877,19997367,2128-02-26 19:00:00,7.643903e+10,pre


#### pro

In [53]:
pro2 = procedureevents.copy()
pro2 = pro2[(pro2['subject_id'].isin(sub_list))]
pro2 = pro2[['subject_id','starttime','endtime','itemid']]
pro2 = pro2.reset_index(drop=True)

# starttime과 endtime을 datetime 형식으로 변환합니다.
pro2['starttime'] = pd.to_datetime(pro2['starttime'])
pro2['endtime'] = pd.to_datetime(pro2['endtime'])

pro2['starttime'] = pro2['starttime'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
pro2['endtime'] = pro2['endtime'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))

In [54]:
# pro2를 리스트로 변환
pro2_list = pro2.values.tolist()
pro2_dic = {}
lst_time = []
lst_itemid = []
lst_subid = []

# pro2_list의 각 원소(즉, 각 절차에 대한 정보)를 순회합니다.
for idx, row in enumerate(tqdm(pro2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    # 시작 날짜와 종료 날짜를 datetime 형식으로 변환
    #start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    #end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
    
    # 날짜 범위 생성
    day_list = date_range(start, end)
    
    # index에 맞춰서 subid, date, itemid를 넣어주기 위한 dic
    pro2_dic[idx] = day_list
    
    # pro2 DATE열에 들어갈 리스트
    lst_time.extend(pro2_dic[idx][:])

    # pro2 itemid열에 들어갈 리스트
    for _ in pro2_dic[idx]:
        lst_itemid.append(itemid)
    
    # pro2 subid열에 들어갈 리스트
    for _ in pro2_dic[idx]:
        lst_subid.append(sub)

100%|█████████████████████████████████| 135528/135528 [00:26<00:00, 5143.95it/s]


In [55]:
pro2sub = pd.DataFrame(lst_subid)
pro2time = pd.DataFrame(lst_time)
pro2item = pd.DataFrame(lst_itemid)

pro2 = pd.concat([pro2sub, pro2time, pro2item], axis = 1)
pro2.columns = ['subject_id', 'charttime', 'itemid']
pro2['type'] = ["pro"]*len(pro2)

In [56]:
pro2

,subject_id,charttime,itemid,type
0,10002013,2160-05-18 14:00:00,225792,pro
1,10002013,2160-05-18 15:00:00,225792,pro
2,10002013,2160-05-18 16:00:00,225792,pro
3,10002013,2160-05-18 17:00:00,225792,pro
4,10002013,2160-05-18 18:00:00,225792,pro
...,...,...,...,...
4005413,19997367,2127-04-06 16:00:00,225202,pro
4005414,19997367,2127-04-06 17:00:00,225202,pro
4005415,19997367,2127-04-06 18:00:00,225202,pro
4005416,19997367,2127-04-06 19:00:00,225202,pro


### merge

In [57]:
m1 = pd.merge(lab2, pre2, on=['subject_id', 'itemid', 'charttime', 'type'], how='outer')

In [58]:
total_data = pd.merge(m1, pro2, on=['subject_id', 'itemid', 'charttime', 'type'], how='outer')

In [59]:
total_data = total_data.sort_values(['subject_id','charttime']).reset_index(drop=True)

In [60]:
total_data = total_data.astype({'itemid':'int'})

In [61]:
total_data

,subject_id,itemid,charttime,flag,type
0,10000826,50809,2146-12-05 11:00:00,abnormal,LAB
1,10000826,50813,2146-12-05 11:00:00,abnormal,LAB
2,10000826,50861,2146-12-05 11:00:00,abnormal,LAB
3,10000826,50862,2146-12-05 11:00:00,abnormal,LAB
4,10000826,50863,2146-12-05 11:00:00,abnormal,LAB
...,...,...,...,...,...
192286628,19997367,51244,2128-10-14 11:00:00,abnormal,LAB
192286629,19997367,51249,2128-10-14 11:00:00,abnormal,LAB
192286630,19997367,51256,2128-10-14 11:00:00,abnormal,LAB
192286631,19997367,51265,2128-10-14 11:00:00,abnormal,LAB


In [62]:
total_data = total_data.drop(['flag', 'type'], axis=1)

total_data = total_data.drop_duplicates(subset=['subject_id', 'itemid', 'charttime'])

In [63]:
# 사전학습 데이터 (전체 데이터)
total_data.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_9)/data.csv', index = False)